In [1]:
import nltk
import gensim
import numpy as np
import pandas as pd
import re,string,unicodedata

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.porter import PorterStemmer
from gensim.utils import simple_preprocess 
from gensim.models import Phrases, Word2Vec
from sklearn.preprocessing import LabelBinarizer
from nltk.tokenize import word_tokenize,sent_tokenize


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
# Load the IMDb dataset from the CSV file
imdb_df = pd.read_csv('/content/dataset.csv', header=None, names=['text', 'humor'])
print(imdb_df.head())

print(imdb_df["humor"].unique())

imdb_df["humor"] = np.where(imdb_df["humor"] == 'False', 0, 1)
print(imdb_df.head())

                                                text  humor
0                                               text  humor
1  Joe biden rules out 2020 bid: 'guys, i'm not r...  False
2  Watch: darvish gave hitter whiplash with slow ...  False
3  What do you call a turtle without its shell? d...   True
4      5 reasons the 2016 election feels so personal  False
['humor' 'False' 'True']
                                                text  humor
0                                               text      1
1  Joe biden rules out 2020 bid: 'guys, i'm not r...      0
2  Watch: darvish gave hitter whiplash with slow ...      0
3  What do you call a turtle without its shell? d...      1
4      5 reasons the 2016 election feels so personal      0


In [30]:
gen_df = pd.read_csv('/content/GEN-sarc-notsarc.csv', header=None, names=['class', 'text'])
gen_df = gen_df.append(pd.read_csv('/content/HYP-sarc-notsarc.csv'))
gen_df = gen_df.append(pd.read_csv('/content/RQ-sarc-notsarc.csv'))

gen_df["class"] = np.where(gen_df["class"] == "notsarc", 0, 1)

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

# Preprocess the dataset
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

<ipython-input-30-e5a98f61f389>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen_df = gen_df.append(pd.read_csv('/content/HYP-sarc-notsarc.csv'))
<ipython-input-30-e5a98f61f389>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen_df = gen_df.append(pd.read_csv('/content/RQ-sarc-notsarc.csv'))


In [31]:
imdb_df.head()

,text,humor
0,text,1
1,"Joe biden rules out 2020 bid: 'guys, i'm not r...",0
2,Watch: darvish gave hitter whiplash with slow ...,0
3,What do you call a turtle without its shell? d...,1
4,5 reasons the 2016 election feels so personal,0


In [32]:
imdb_df['text']=imdb_df['text'].apply(denoise_text)
gen_df['text']=gen_df['text'].apply(denoise_text)

<ipython-input-30-e5a98f61f389>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [33]:
words, imdb_words, gen_words = [], [], []
for i in imdb_df.text.values:
    words.append(i.split())
    imdb_words.append(i.split())
words=words[1:]

for i in gen_df.text.values:
    words.append(i.split())
    gen_words.append(i.split())

In [34]:
len(words)

209387

In [35]:
imdb_wv_model = Word2Vec(words, vector_size=500, window=3, min_count=1, workers=16)

In [36]:
print(imdb_wv_model)

Word2Vec<vocab=178665, vector_size=500, alpha=0.025>


In [37]:
print(len(words))

209387


In [38]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(178665)
tokenizer.fit_on_texts(words)
text = tokenizer.texts_to_sequences(words)
text = keras.utils.pad_sequences(text, 50)

print(text[:5])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0   916  1884   636  3927 69516 69517
      8 48343]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0   985 69518   440 32225 48344
   1187  4433]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     2  4502   108
  27962  1913]
 [    0     0     0     0     0     0     0     0     0     0     0

In [39]:
y = imdb_df["humor"]

In [40]:
all(y)

False

In [41]:
y[:5]

0    1
1    0
2    0
3    1
4    0
Name: humor, dtype: int64

In [42]:
txt = tokenizer.texts_to_sequences(imdb_words)
txt = keras.utils.pad_sequences(txt, 50)

txt[:2]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,  1422],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,   916,  1884,   636,
         3927, 69516, 69517,     8, 48343]], dtype=int32)

In [43]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(np.array(txt), np.array(y), train_size=0.8, stratify=y)

In [44]:
import tensorflow as tf

model = keras.models.Sequential([

    # Embedding layer
    keras.layers.Embedding(input_dim=len(imdb_wv_model.wv.key_to_index),
                            output_dim=500,
                            input_length=50,
                            weights=[imdb_wv_model.wv.vectors],
                            trainable=True),
    
    keras.layers.Dropout(0.2),

    # BiLSTM layer
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),

    keras.layers.Bidirectional(keras.layers.LSTM(64)),

    keras.layers.Dropout(0.2),

    keras.layers.Dense(200),

    keras.layers.Activation('relu'),

    keras.layers.Dropout(0.2),
    
    # Output layer
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 500)           89332500  
                                                                 
 dropout (Dropout)           (None, 50, 500)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 50, 256)          644096    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 200)               2

In [ ]:
val_recall = 0.8160
val_precision = 0.8572
val_f1 = (2 * val_precision * val_recall) / (val_precision + val_recall)

val_f1

0.8360927563949319

In [45]:
txt2 = tokenizer.texts_to_sequences(gen_words)
txt2 = keras.utils.pad_sequences(txt2, 50)

y = gen_df["class"]

x_train, x_test, y_train, y_test = train_test_split(np.array(txt2), np.array(y), train_size=0.8, stratify=y)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

Epoch 1/10
235/235 [==============================] - 41s 159ms/step - loss: 0.4683 - accuracy: 0.8442 - precision: 0.8476 - recall: 0.9950 - val_loss: 0.4066 - val_accuracy: 0.8472 - val_precision: 0.8472 - val_recall: 1.0000
Epoch 2/10
235/235 [==============================] - 20s 84ms/step - loss: 0.2552 - accuracy: 0.8992 - precision: 0.9131 - recall: 0.9738 - val_loss: 0.5582 - val_accuracy: 0.7481 - val_precision: 0.8678 - val_recall: 0.8290
Epoch 3/10
235/235 [==============================] - 15s 61ms/step - loss: 0.0833 - accuracy: 0.9730 - precision: 0.9840 - recall: 0.9841 - val_loss: 0.6831 - val_accuracy: 0.7322 - val_precision: 0.8632 - val_recall: 0.8127
Epoch 4/10
235/235 [==============================] - 11s 48ms/step - loss: 0.0460 - accuracy: 0.9851 - precision: 0.9942 - recall: 0.9882 - val_loss: 0.7998 - val_accuracy: 0.7567 - val_precision: 0.8584 - val_recall: 0.8536
Epoch 5/10
235/235 [==============================] - 10s 41ms/step - loss: 0.0352 - accuracy: 

In [46]:
val_recall = 0.8646 *100
val_precision = 0.8190 *100
val_f1 = (2 * val_precision * val_recall) / (val_precision + val_recall)

val_f1

84.1182466143977

In [47]:
model.save("/bilstm-with-intermediate-task-humor.h")

In [48]:
gen_wv_model = Word2Vec(gen_words, vector_size=500, window=3, min_count=1, workers=16)

In [49]:
print(gen_wv_model)

Word2Vec<vocab=46908, vector_size=500, alpha=0.025>


In [50]:
# tokenizer = Tokenizer(46908)
# tokenizer.fit_on_texts(gen_words)
txt2 = tokenizer.texts_to_sequences(gen_words)
txt2 = keras.utils.pad_sequences(txt2, 50)

model = keras.models.Sequential([

    # Embedding layer
    keras.layers.Embedding(input_dim=len(imdb_wv_model.wv.key_to_index),
                            output_dim=500,
                            input_length=50,
                            weights=[imdb_wv_model.wv.vectors],
                            trainable=True),
    
    keras.layers.Dropout(0.2),

    # BiLSTM layer
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),

    keras.layers.Bidirectional(keras.layers.LSTM(64)),

    keras.layers.Dropout(0.2),

    keras.layers.Dense(200),

    keras.layers.Activation('relu'),

    keras.layers.Dropout(0.2),
    
    # Output layer
    keras.layers.Dense(1, activation='sigmoid')
])


y = gen_df["class"]

x_train, x_test, y_train, y_test = train_test_split(np.array(txt2), np.array(y), train_size=0.8, stratify=y, random_state=42)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
118/118 [==============================] - 35s 223ms/step - loss: 0.4235 - accuracy: 0.8472 - precision_1: 0.8474 - recall_1: 0.9998 - val_loss: 0.4095 - val_accuracy: 0.8472 - val_precision_1: 0.8472 - val_recall_1: 1.0000
Epoch 2/10
118/118 [==============================] - 17s 143ms/step - loss: 0.4073 - accuracy: 0.8474 - precision_1: 0.8474 - recall_1: 1.0000 - val_loss: 0.4131 - val_accuracy: 0.8472 - val_precision_1: 0.8472 - val_recall_1: 1.0000
Epoch 3/10
118/118 [==============================] - 14s 116ms/step - loss: 0.2167 - accuracy: 0.9161 - precision_1: 0.9296 - recall_1: 0.9749 - val_loss: 0.5578 - val_accuracy: 0.8078 - val_precision_1: 0.8571 - val_recall_1: 0.9277
Epoch 4/10
118/118 [==============================] - 10s 85ms/step - loss: 0.0568 - accuracy: 0.9836 - precision_1: 0.9915 - recall_1: 0.9892 - val_loss: 0.6862 - val_accuracy: 0.7923 - val_precision_1: 0.8628 - val_recall_1: 0.8975
Epoch 5/10
118/118 [==============================] - 9s 77ms

In [51]:
val_recall = 0.8716 *100
val_precision = 0.6826 *100
val_f1 = (2 * val_precision * val_recall) / (val_precision + val_recall)

val_f1

76.5608235748295

In [ ]:
!zip -r "/bilstm.zip" "/bert-with-intermediate-task.h"

  adding: bert-with-intermediate-task.h/ (stored 0%)
  adding: bert-with-intermediate-task.h/assets/ (stored 0%)
  adding: bert-with-intermediate-task.h/keras_metadata.pb (deflated 92%)
  adding: bert-with-intermediate-task.h/variables/ (stored 0%)
  adding: bert-with-intermediate-task.h/variables/variables.index (deflated 67%)
  adding: bert-with-intermediate-task.h/variables/variables.data-00000-of-00001 (deflated 48%)
  adding: bert-with-intermediate-task.h/fingerprint.pb (stored 0%)
  adding: bert-with-intermediate-task.h/saved_model.pb (deflated 90%)
